# Cross-validation methods

In this notebook you will consider K-Fold cross validation method to estimate quality of a claasifier.

In [34]:
%matplotlib inline
import pandas as pd
import numpy as np
import numpy.testing as np_testing
import matplotlib.pyplot as plt

# Load MAGIC Data Set

<center><img src="https://github.com/HSE-LAMBDA/ML-IDS/blob/main/course_1/4_topic/4_uncertainty_estimation/img/magic1.png?raw=1" width="1000"></center>

Source: https://magic.mpp.mpg.de/

In [35]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data

--2021-11-24 14:05:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1477391 (1.4M) [application/x-httpd-php]
Saving to: ‘magic04.data.1’

magic04.data.1      100%[===================>]   1.41M  7.43MB/s    in 0.2s    

2021-11-24 14:05:18 (7.43 MB/s) - ‘magic04.data.1’ saved [1477391/1477391]



Features description:
- **Length:** continuous # major axis of ellipse [mm]
- **Width:** continuous # minor axis of ellipse [mm]
- **Size:** continuous # 10-log of sum of content of all pixels [in #phot]
- **Conc:** continuous # ratio of sum of two highest pixels over fSize [ratio]
- **Conc1:** continuous # ratio of highest pixel over fSize [ratio]
- **Asym:** continuous # distance from highest pixel to center, projected onto major axis [mm]
- **M3Long:** continuous # 3rd root of third moment along major axis [mm]
- **M3Trans:** continuous # 3rd root of third moment along minor axis [mm]
- **Alpha:** continuous # angle of major axis with vector to origin [deg]
- **Dist:** continuous # distance from origin to center of ellipse [mm]
- **Label:** g,h # gamma (signal), hadron (background)

g = gamma (signal): 12332 \
h = hadron (background): 6688

In [36]:
f_names = np.array(["Length", "Width", "Size", "Conc", "Conc1", "Asym", "M3Long", "M3Trans", "Alpha", "Dist"])

data = pd.read_csv("magic04.data", header=None, names=list(f_names)+["Label"])
data.head()

,Length,Width,Size,Conc,Conc1,Asym,M3Long,M3Trans,Alpha,Dist,Label
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


# Data preparation

In [37]:
# prepare a matrix of input features
X = data[f_names].values

# prepare a vector of true labels
y = 1 * (data['Label'].values == "g")

In [38]:
# print sizes of X and y
X.shape, y.shape

((19020, 10), (19020,))

In [39]:
X[:2]

array([[ 2.87967e+01,  1.60021e+01,  2.64490e+00,  3.91800e-01,
         1.98200e-01,  2.77004e+01,  2.20110e+01, -8.20270e+00,
         4.00920e+01,  8.18828e+01],
       [ 3.16036e+01,  1.17235e+01,  2.51850e+00,  5.30300e-01,
         3.77300e-01,  2.62722e+01,  2.38238e+01, -9.95740e+00,
         6.36090e+00,  2.05261e+02]])

In [40]:
y[:5]

array([1, 1, 1, 1, 1])

# Preprocessing

Scale input data using StandardScaler:
$$
X_{new} = \frac{X - \mu}{\sigma}
$$

In [41]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Create object of the class and set up its parameters
ss = StandardScaler()

# Estimate mean and sigma values
ss.fit(X)

# Scale the sample
X = ss.transform(X)

# Train / test split

Split the dataset into train and test samples. 

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=11)

# Define a model


Now let's create a neural network and fit it.

In [43]:
!pip install pytorch-lightning 

In [44]:
import torch
from torch.nn import functional as F
from torch import nn
import pytorch_lightning as pl

class Model(pl.LightningModule):

    def __init__(self):
        super().__init__()
        
        # define all layers of the netwrok
        self.net = nn.Sequential(
                                nn.Linear(10, 10), 
                                nn.Tanh(), 
                                nn.Linear(10, 1), 
                                nn.Sigmoid())

    
    def forward(self, x):
        # make a prediction for x
        return self.net(x)

    # calculate loss function values
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss

    # define optimizer to fit the network
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [45]:
model = Model()
model

Model(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

# Data loader creation

We will define a helping function for converting `X_train` and `y_train` into PyTorch `DataLoader`.

In [46]:
from torch.utils.data import TensorDataset, DataLoader

def create_data_loader(X_train, y_train, batch_size=128):
    # combine X and y into one pytorch tensor dataset
    dataset_train = TensorDataset(torch.tensor(X_train, dtype=torch.float), 
                                  torch.tensor(y_train.reshape(-1, 1), dtype=torch.float))
    # loader divides our train data into batches
    train_loader = DataLoader(dataset_train, batch_size=batch_size, num_workers=4)
    return train_loader

In [47]:
# example of usage
create_data_loader(X[:5], y[:5], 1)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Quality metrics

We will use the function below to calculate quality metrics for out model.

In [48]:
from sklearn import metrics

def quality_metrics_report(y_true, y_pred, y_proba):
    """
    Parameters
    ----------
    y_true: array-like of shape (n_samples,)
        Ground truth (correct) target values.
    y_pred: array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    y_proba : array, shape = [n_samples]
        Target scores, can be probability estimates of the positive
        class.
        
    Returns
    -------
    List of metric values: [accuracy, precision, recall, f1, roc_auc]
    """
    
    accuracy  = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall    = metrics.recall_score(y_true, y_pred)
    f1        = metrics.f1_score(y_true, y_pred)
    roc_auc   = metrics.roc_auc_score(y_true, y_proba)
    
    return [accuracy, precision, recall, f1, roc_auc]

In [49]:
# example of usage
quality_metrics_report(y_true=[0, 0, 1, 1], 
                       y_pred=[0, 1, 1, 1], 
                       y_proba=[0.1, 0.6, 0.8, 0.9])

[0.75, 0.6666666666666666, 1.0, 0.8, 1.0]

# Fit model

In [50]:
# init model and trainer
model = Model()
trainer = pl.Trainer(max_epochs=10)

# create pytorch dataloader
train_loader = create_data_loader(X_train, y_train, batch_size=128)

# fit the model
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 121   
------------------------------------
121       Trainable params
0         Non-trainable params
121       Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

# Test model

In [51]:
# make prediction
y_test_proba = model(torch.tensor(X_test, dtype=torch.float))[:, 0].detach().numpy()
y_test_pred  = 1 * (y_test_proba > 0.5)

In [52]:
# example
y_test_proba[:5]

array([0.9615558 , 0.00659889, 0.77472204, 0.73349035, 0.93751407],
      dtype=float32)

In [53]:
# example
y_test_pred[:5]

array([1, 0, 1, 1, 1])

In [54]:
# example
y_test[:5]

array([0, 0, 1, 0, 1])

In [55]:
# compute quaility metrics
report = quality_metrics_report(y_test, y_test_pred, y_test_proba)
report

[0.8583596214511041,
 0.857333531069257,
 0.9375608173856633,
 0.8956541947478504,
 0.9145969921874395]

In [56]:
# transform report into a table
df = pd.DataFrame()
df['Metrics'] = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
df['Mean']    = report
df

,Metrics,Mean
0,Accuracy,0.858360
1,Precision,0.857334
2,Recall,0.937561
3,F1,0.895654
4,ROC AUC,0.914597


# Uncertainty estimation with bootstrap

We will estimate uncertainties of the quality metrics using the bootstrap method.

<center><img src="https://github.com/HSE-LAMBDA/ML-IDS/blob/main/course_1/4_topic/4_uncertainty_estimation/img/bootstrap.png?raw=1" width="600"></center>

Algorithm:
    
- Given a model fitted on a train sample
- N – number of objects in a test sample
- For i=1, …, B do
    - Sample with **replacement** a subsample with N objects from the test sample
    - Calculate quality metrics on this subsample
- Estimate statistics of the metrics: mean, **variance**, confidence intervals


# Task 1
Using `y_test_pred`, `y_test_proba` predictions, and the bootstrap estimate means and standard deviation of the quality metrics for the classifier above. 

**Hint:** use `resample(y_test, y_test_pred, y_test_proba, replace=True, n_samples=len(y_test))` to make bootstrap samples. Use function `quality_metrics_report` above to compute the quality metrics.

In [63]:
from sklearn.utils import resample

def boostrap_uncertainties(y_test, y_test_pred, y_test_proba, n_iterations=100):
    
    metrics = []
    
    # go through each iteration of KFold
    for it in range(n_iterations):
        
        ### BEGIN SOLUTION
        boot_y_test, boot_y_test_pred, boot_y_test_proba = resample(y_test, y_test_pred, y_test_proba, replace=True, n_samples=len(y_test))
        metrics_iter = quality_metrics_report(boot_y_test, boot_y_test_pred, boot_y_test_proba)
        metrics.append(metrics_iter)
        ### END SOLUTION
        
    metrics = np.array(metrics)
    df = pd.DataFrame()
    df['Metrics'] = columns=['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
    df['Mean']    = metrics.mean(axis=0)
    df['Std']     = metrics.std(axis=0)
    
    return df

In [64]:
# run uncertainty estimation
report = boostrap_uncertainties(y_test, y_test_pred, y_test_proba, n_iterations=100)
report

,Metrics,Mean,Std
0,Accuracy,0.858506,0.003767
1,Precision,0.857178,0.004228
2,Recall,0.937750,0.003102
3,F1,0.895649,0.002920
4,ROC AUC,0.914459,0.003078


Expected approximate output:

<center>   
    
```python
Metrics       Mean     Std
0 Accuracy    0.862501 0.003302
1 Precision   0.861022 0.003830
2 Recall      0.939593 0.003435
3 F1          0.898587 0.002760
``` 
    
</center>

In [ ]:
### BEGIN HIDDEN TESTS
actual  = report.values[-1, 1:].astype(np.float)
desired = np.array([0.920540, 0.002441])
np_testing.assert_allclose(actual, desired, atol=0.005)
### END HIDDEN TESTS